In [9]:


import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_66.jpg
/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_21.jpg
/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_64.jpg
/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_33.jpg
/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_51.jpg
/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_61.jpg
/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_9.jpg
/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_38.jpg
/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_58.jpg
/kaggle/input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_72.jpg
/kaggle/inp

In [10]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
from tensorflow.keras.preprocessing import image

In [11]:
train_path= "../input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/train"
test_path="../input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation"

In [12]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [13]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'binary')
validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size = (224,224),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary')

Found 1400 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [14]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(64,activation='relu'))
model.add(GaussianNoise(0.25))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 53, 53, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 51, 51, 128)      

In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','AUC','Precision','Recall'])

In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
es=EarlyStopping(patience=3,monitor='val_loss')
filepath='best_model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch= 50,
    callbacks=checkpoint
    )

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
model.evaluate(train_generator)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
img = image.load_img("../input/breast-cancer-patients-mris/Breast Cancer Patients MRI's/validation/Healthy/S_33.jpg",target_size=(224,224))
imag = image.img_to_array(img)
imaga = np.expand_dims(imag,axis=0) 
ypred = model.predict(imaga)
print(ypred)
a=ypred[0]
if a<0.5:
      op="Healthy"   
else:
      op="Cancer"
plt.imshow(img)
print("THE UPLOADED IMAGE IS SUSPECTED AS: "+str(op)) 